# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [4]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint


## Data Importing

In [ ]:
## Data Importing load one file first to see what type of data you're dealing with and what attributes it has 
# loop over all files and put them into a dataframe 

In [5]:
# 1.1 locate files
data_dir = "../data"
files = os.listdir(data_dir)
json_files = [f for f in files if f.endswith(".json")]

FileNotFoundError: [Errno 2] No such file or directory: '../data'

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [ ]:
print(df.shape)
print(df.columns)
print(df.dtypes)
df.info()
df.describe(include='all')


### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
df_hashed = df.copy().astype(str)  # Updated to avoid the `applymap` warning
duplicates_mask = df_hashed.duplicated()
duplicates = df[duplicates_mask]

duplicate_count = df_hashed.duplicated().sum()

# If you want to see them:
if duplicate_count > 0:
    duplicate_rows = df[df_hashed.duplicated()]
    print("\nDuplicate rows:")
    display(duplicate_rows)

In [ ]:
# All duplicate rows (excluding firsts)
dup_df = df_hashed[df_hashed.duplicated(keep=False)]

# Columns where all duplicate rows have the same value
constant_cols = [col for col in df_hashed.columns if dup_df[col].nunique() == 1]

print("\nColumns that are identical in all duplicate rows:")
for col in constant_cols:
    print(f"  {col}")

In [ ]:
# OHE categorical variables/ tags here
# tags will have to be done manually


import pandas as pd

def flatten_nested_columns(df):
    # 1. Flatten 'tags' - simple list of strings
    if 'tags' in df.columns:
        df['tags_flat'] = df['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
        df.drop(columns=['tags'], inplace=True)

    # 2. Flatten 'branding' - extract 'name' field
    if 'branding' in df.columns:
        df['branding_names'] = df['branding'].apply(
            lambda lst: ', '.join([str(d.get('name', '')) for d in lst if isinstance(d, dict)]) if isinstance(lst, list) else ''
        )
        df.drop(columns=['branding'], inplace=True)

    # 3. Flatten 'photos' - count of photos or first image href
    if 'photos' in df.columns:
        df['photo_count'] = df['photos'].apply(lambda x: len(x) if isinstance(x, list) else 0)
        df['first_photo_href'] = df['photos'].apply(
            lambda x: x[0].get('href', '') if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else ''
        )
        df.drop(columns=['photos'], inplace=True)

    # 4. Flatten 'virtual_tours' - first link if available
    if 'virtual_tours' in df.columns:
        df['first_virtual_tour'] = df['virtual_tours'].apply(
            lambda x: x[0].get('href', '') if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else ''
        )
        df.drop(columns=['virtual_tours'], inplace=True)

    # 5. Flatten 'source.agents' - get all 'office_name' entries
    if 'source.agents' in df.columns:
        df['source.agents.office_names'] = df['source.agents'].apply(
            lambda agents: ', '.join([str(a.get('office_name', '') or '') for a in agents]) if isinstance(agents, list) else ''
        )
        df.drop(columns=['source.agents'], inplace=True)

    # 6. Flatten 'other_listings.rdc' - get count or listing_ids
    if 'other_listings.rdc' in df.columns:
        df['other_listings_count'] = df['other_listings.rdc'].apply(
            lambda x: len(x) if isinstance(x, list) else 0
        )
        df['other_listing_ids'] = df['other_listings.rdc'].apply(
            lambda x: ', '.join([str(i.get('listing_id', '')) for i in x]) if isinstance(x, list) else ''
        )
        df.drop(columns=['other_listings.rdc'], inplace=True)

    # 7. Flatten 'community.advertisers' - extract phone numbers if present
    if 'community.advertisers' in df.columns:
        def extract_phone(advertisers):
            if isinstance(advertisers, list):
                phones = []
                for advertiser in advertisers:
                    office = advertiser.get('office', {}) if isinstance(advertiser, dict) else {}
                    phone_list = office.get('phones', [])
                    for phone in phone_list:
                        phones.append(phone.get('number', ''))
                return ', '.join(filter(None, phones))
            return ''
        
        df['advertiser_phones'] = df['community.advertisers'].apply(extract_phone)
        df.drop(columns=['community.advertisers'], inplace=True)

    return df


### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [ ]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized